In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.isna().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [5]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show'],
      dtype='object')

In [6]:
y = df['No-show']
X = df[['Gender','Age','Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']]

In [7]:
X

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,F,62,0,1,0,0,0,0
1,M,56,0,0,0,0,0,0
2,F,62,0,0,0,0,0,0
3,F,8,0,0,0,0,0,0
4,F,56,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...
110522,F,56,0,0,0,0,0,1
110523,F,51,0,0,0,0,0,1
110524,F,21,0,0,0,0,0,1
110525,F,38,0,0,0,0,0,1


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder

enc = LabelEncoder()
y = enc.fit_transform(y)

enc = OrdinalEncoder()
X.loc[:,['Gender']] = enc.fit_transform(X[['Gender']])

sca = StandardScaler()
X.loc[:,['Age']] = sca.fit_transform(X[['Age']].astype(float))



/var/folders/f9/84hvpcld76d5cmvtgy70zd300000gn/T/ipykernel_18329/2259213352.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.07793239  0.81830565  1.07793239 ... -0.69618366  0.03942544
  0.73176341]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:,['Age']] = sca.fit_transform(X[['Age']].astype(float))


In [9]:
X

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0.0,1.077932,0,1,0,0,0,0
1,1.0,0.818306,0,0,0,0,0,0
2,0.0,1.077932,0,0,0,0,0,0
3,0.0,-1.258708,0,0,0,0,0,0
4,0.0,0.818306,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...
110522,0.0,0.818306,0,0,0,0,0,1
110523,0.0,0.601950,0,0,0,0,0,1
110524,0.0,-0.696184,0,0,0,0,0,1
110525,0.0,0.039425,0,0,0,0,0,1


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)


In [ ]:
models = {}

# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [2, 3, 5, 7, 9, 11],
            #'max_features': [1,2,3,4,5,'sqrt', 'log2']

        }
model = DecisionTreeClassifier(random_state=0)

grid_clf = GridSearchCV(model, param_grid, cv=10, scoring='recall')
grid_clf.fit(X_train, y_train)
model = grid_clf.best_estimator_

y_pred = model.predict(X_test)

from sklearn.metrics import recall_score, confusion_matrix
print(model)
score = recall_score(y_test, y_pred)
print('recall score: '+ str(score))
print(confusion_matrix(y_test,y_pred))
models['DecisionTree']=[model,y_pred]

DecisionTreeClassifier(max_depth=2, random_state=0)
recall score: 0.0
[[17725     0]
 [ 4381     0]]


In [19]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

param_grid = {
            'n_estimators': [7,10,12, 15, 20, 25, 35, 50],
            'max_depth': [2, 3, 5, 7, 9, 11],
            'max_features': ['sqrt', 'log2'],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'min_samples_split': [2,3,4,5]
        }

model2 = RandomForestClassifier(n_estimators=20, criterion='gini', random_state=0)

grid_clf = GridSearchCV(model2, param_grid, cv=10, scoring='f1')
grid_clf.fit(X_train, y_train)
model2 = grid_clf.best_estimator_


y_pred2 = model2.predict(X_test)

from sklearn.metrics import recall_score, confusion_matrix
score = recall_score(y_test, y_pred2)
print('recall score: '+ str(score))
print(confusion_matrix(y_test,y_pred2))
models['RandomForest']=[model2,y_pred2]

recall score: 0.0066194932663775395
[[17671    54]
 [ 4352    29]]


/Users/koa/Documents/ML Course/medical appt no shows/.venv/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [24]:
def export_to_pickle(models_dict,X_train, X_test, y_train, y_test, X, y):
    import pickle
    path = 'models'
    for key in models_dict.keys():
        pickle.dump(models_dict[key][0], open('models/'+key+'.pkl', 'wb'))

    dfs = [X_train, X_test, y_train, y_test, X, y]
    pickle.dump(dfs, open('models/dataframes.pkl', 'wb'))



export_to_pickle(models,X_train, X_test, y_train, y_test, X, y)

In [21]:
model2.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(max_depth=11, max_features='log2', min_samples_split=5,
                       n_estimators=7, random_state=0)>

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

param_grid = {
            'n_estimators': [6,7,10,12, 15, 20],
            'max_depth': [13],
            'max_features': [4],
            'criterion': ['log_loss'],
            'min_samples_split': [3,4,5,6,7]
        }

model2 = RandomForestClassifier(n_estimators=20, criterion='gini', random_state=0)

grid_clf = GridSearchCV(model2, param_grid, cv=10, scoring='f1')
grid_clf.fit(X_train, y_train)
model2 = grid_clf.best_estimator_


y_pred2 = model2.predict(X_test)

from sklearn.metrics import recall_score, confusion_matrix
score = recall_score(y_test, y_pred2)
print('recall score: '+ str(score))
print(confusion_matrix(y_test,y_pred2))
models['RandomForest2']=[model2,y_pred2]

recall score: 0.011869436201780416
[[17617   108]
 [ 4329    52]]


/Users/koa/Documents/ML Course/medical appt no shows/.venv/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [25]:
model2.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(criterion='log_loss', max_depth=13, max_features=4,
                       min_samples_split=4, n_estimators=6, random_state=0)>